# Identificazione delle "Golden Labels" per il Test Set - 7 Classi

**Obiettivo dello script:**
Questo script adatta il processo di identificazione delle "golden labels" per il sistema a 7 classi di emozioni. Confronta il dataset ASLLRP classificato con 7 classi con il dataset EmoSign per creare un test set affidabile e un training/validation set separato.

### Le 7 Classi di Emozioni:

1. **Estremamente Negativo** (Strongly Negative / Extremely Negative)
2. **Negativo** (Negative)
3. **Un po' Negativo** (Somewhat Negative)
4. **Neutro** (Neutral)
5. **Un po' Positivo** (Somewhat Positive)
6. **Positivo** (Positive)
7. **Estremamente Positivo** (Strongly Positive / Extremely Positive)

**Funzionamento dello script:**

1.  **Caricamento dei Dataset:**

    - **Dataset EmoSign (da Hugging Face):** Ground truth validato manualmente
    - **Dataset Locale (7 classi):** File `asllrp_video_sentiment_data_7_classes.csv` generato nel notebook precedente

2.  **Estrazione degli ID Comuni:**

    - Estrae l'`utterance_id` univoco da entrambi i dataset
    - Trova l'intersezione tra i due insiemi di ID

3.  **Identificazione delle "Golden Labels":**

    - Le corrispondenze trovate costituiscono le **"golden labels"**
    - Queste sono etichette affidabili, validate da una fonte esterna

4.  **Creazione e Salvataggio dei Dataset:**
    - **Test Set (Golden Labels):** Utilizzato esclusivamente per valutazione finale
    - **Training/Validation Set:** Tutti gli altri dati, per addestramento e validazione
    - Garantisce separazione netta tra train e test set

### Importanza delle Golden Labels:

- **Affidabilità:** Confermate da studi precedenti (EmoSign)
- **Comparabilità:** Permettono di confrontare risultati con altri studi
- **Validazione robusta:** Test set di alta qualità per valutazione finale


## Fase 1: Importazione delle Librerie


In [1]:
import pandas as pd
import os
from collections import Counter

## Fase 2: Caricamento del Dataset EmoSign da Hugging Face


In [2]:
# Caricamento del dataset EmoSign da Hugging Face
print("Caricamento del dataset EmoSign da Hugging Face...")
print("Nota: Potrebbe essere necessario autenticarsi con 'huggingface-cli login'\n")

try:
    df_emosign = pd.read_csv("hf://datasets/catfang/emosign/emosign_dataset.csv")
    print("✅ Dataset EmoSign caricato con successo.")

    # Estrai l'ID utterance dalla colonna 'video_name'
    # L'ID è la parte numerica dopo il trattino basso '_'
    df_emosign["utterance_id"] = (
        df_emosign["video_name"].str.split("_").str[-1].astype(str)
    )

    emosign_ids = set(df_emosign["utterance_id"])
    print(f"Trovati {len(emosign_ids)} ID unici nel dataset EmoSign.")

    # Mostra alcune informazioni sul dataset
    print(f"\nColonne EmoSign: {list(df_emosign.columns)}")
    print(f"Numero totale di righe: {len(df_emosign)}")

except Exception as e:
    print(f"❌ Errore durante il caricamento del dataset da Hugging Face: {e}")
    print("\n💡 Suggerimenti:")
    print("1. Esegui 'huggingface-cli login' nel terminale")
    print("2. Verifica la connessione internet")
    print("3. Controlla che il dataset 'catfang/emosign' sia accessibile")
    emosign_ids = set()
    df_emosign = pd.DataFrame()

Caricamento del dataset EmoSign da Hugging Face...
Nota: Potrebbe essere necessario autenticarsi con 'huggingface-cli login'



/Users/ignazioemanuelepicciche/Documents/TESI Magistrale UCBM/Improved_EmoSign_Thesis/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Dataset EmoSign caricato con successo.
Trovati 200 ID unici nel dataset EmoSign.

Colonne EmoSign: ['video_name', 'Sentiment', 'joy', 'excited', 'surprise_pos', 'surprise_neg', 'worry', 'sadness', 'fear', 'disgust', 'frustration', 'anger', 'Reasoning_1', 'Reasoning_2', 'Reasoning_3', 'utterance_id']
Numero totale di righe: 200


## Fase 3: Caricamento del Dataset Locale (7 Classi)


In [3]:
# Caricamento del dataset locale con 7 classi
if emosign_ids:
    print("\nCaricamento del dataset locale con sentiment a 7 classi...")
    local_csv_path = "../../data/processed/asllrp_video_sentiment_data_7_classes.csv"

    try:
        df_local = pd.read_csv(local_csv_path)
        print("✅ Dataset locale caricato con successo.")

        # Estrai l'ID utterance dalla colonna 'video_name'
        # Rimuovi l'estensione '.mp4'
        df_local["utterance_id"] = (
            df_local["video_name"].str.replace(".mp4", "", regex=False).astype(str)
        )

        local_ids = set(df_local["utterance_id"])
        print(f"Trovati {len(local_ids)} ID unici nel dataset locale.")

        # Mostra informazioni sul dataset
        print(f"\nColonne dataset locale: {list(df_local.columns)}")
        print(f"Numero totale di righe: {len(df_local)}")

        # Mostra la distribuzione delle classi
        print("\nDistribuzione delle emozioni nel dataset locale:")
        emotion_dist = df_local["emotion"].value_counts().sort_index()
        for emotion, count in emotion_dist.items():
            percentage = (count / len(df_local)) * 100
            print(f"  {emotion:25s}: {count:5d} ({percentage:5.2f}%)")

    except FileNotFoundError:
        print(f"❌ Errore: File non trovato a '{local_csv_path}'")
        print(
            "\n💡 Assicurati di aver eseguito il notebook '02_sentiment_analysis_ASLLRP_7_classes.ipynb' prima."
        )
        df_local = pd.DataFrame()
        local_ids = set()
    except Exception as e:
        print(f"❌ Errore durante il caricamento: {e}")
        df_local = pd.DataFrame()
        local_ids = set()
else:
    print("\n⚠️  Salto il caricamento del dataset locale a causa di errori precedenti.")
    df_local = pd.DataFrame()
    local_ids = set()


Caricamento del dataset locale con sentiment a 7 classi...
✅ Dataset locale caricato con successo.
Trovati 2118 ID unici nel dataset locale.

Colonne dataset locale: ['video_name', 'caption', 'compound_score', 'emotion', 'utterance_id']
Numero totale di righe: 2118

Distribuzione delle emozioni nel dataset locale:
  Extremely Negative       :     9 ( 0.42%)
  Extremely Positive       :    68 ( 3.21%)
  Negative                 :   183 ( 8.64%)
  Neutral                  :  1007 (47.54%)
  Positive                 :   435 (20.54%)
  Somewhat Negative        :   202 ( 9.54%)
  Somewhat Positive        :   214 (10.10%)


## Fase 4: Identificazione delle Corrispondenze ("Golden Labels")


In [4]:
# Trova le corrispondenze tra i due dataset
if not df_local.empty and emosign_ids:
    # Trova gli ID in comune
    golden_ids = local_ids.intersection(emosign_ids)

    print(f"\n{'='*60}")
    print(f"IDENTIFICAZIONE GOLDEN LABELS")
    print(f"{'='*60}")
    print(f"\nID nel dataset EmoSign: {len(emosign_ids)}")
    print(f"ID nel dataset locale: {len(local_ids)}")
    print(f"\n✨ GOLDEN LABELS TROVATE: {len(golden_ids)}")
    print(f"{'='*60}")

    # Calcola percentuali
    percentage_of_local = (len(golden_ids) / len(local_ids)) * 100 if local_ids else 0
    percentage_of_emosign = (
        (len(golden_ids) / len(emosign_ids)) * 100 if emosign_ids else 0
    )

    print(f"\nPercentuale rispetto al dataset locale: {percentage_of_local:.2f}%")
    print(f"Percentuale rispetto al dataset EmoSign: {percentage_of_emosign:.2f}%")

    # Filtra il DataFrame locale per mantenere solo le golden labels
    golden_labels_df = df_local[df_local["utterance_id"].isin(golden_ids)].copy()

    # Mostra la distribuzione delle emozioni nelle golden labels
    print("\nDistribuzione delle emozioni nelle Golden Labels:")
    golden_emotion_dist = golden_labels_df["emotion"].value_counts().sort_index()
    for emotion, count in golden_emotion_dist.items():
        percentage = (count / len(golden_labels_df)) * 100
        print(f"  {emotion:25s}: {count:5d} ({percentage:5.2f}%)")

    print("\nPrime 5 golden labels:")
    print(
        golden_labels_df[["video_name", "caption", "compound_score", "emotion"]].head()
    )

else:
    print(
        "\n❌ Impossibile trovare le corrispondenze a causa di errori nel caricamento dei dati."
    )
    golden_labels_df = pd.DataFrame()
    golden_ids = set()


IDENTIFICAZIONE GOLDEN LABELS

ID nel dataset EmoSign: 200
ID nel dataset locale: 2118

✨ GOLDEN LABELS TROVATE: 200

Percentuale rispetto al dataset locale: 9.44%
Percentuale rispetto al dataset EmoSign: 100.00%

Distribuzione delle emozioni nelle Golden Labels:
  Extremely Negative       :     5 ( 2.50%)
  Extremely Positive       :    30 (15.00%)
  Negative                 :    78 (39.00%)
  Positive                 :    71 (35.50%)
  Somewhat Negative        :    16 ( 8.00%)

Prime 5 golden labels:
      video_name                                            caption  \
26  83664512.mp4  I was like, “Oh, wow, that`s fine.” He wanted ...   
31     25328.mp4  There are so many things that are wrong with t...   
38    253715.mp4  Then they would ask the HIT group about the ca...   
39    255438.mp4  Or they will ask another group SMASHED about t...   
40    253816.mp4  Or ask the COLLIDED group about the car and if...   

    compound_score   emotion  
26          0.7430  Positive  
31

## Fase 5: Creazione del Test Set (Golden Labels)


In [5]:
# Salva le golden labels come test set
if not golden_labels_df.empty:
    # Rimuovi la colonna 'utterance_id' di supporto
    golden_labels_output = golden_labels_df.drop(columns=["utterance_id"]).copy()

    # Definisci il percorso di output
    output_path = "../../data/processed/golden_label_sentiment_7_classes.csv"

    # Crea la directory se non esiste
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Salva il file
    golden_labels_output.to_csv(output_path, index=False)

    print(f"\n{'='*60}")
    print(f"TEST SET SALVATO")
    print(f"{'='*60}")
    print(f"\n✅ File salvato: {output_path}")
    print(f"Numero di righe: {len(golden_labels_output)}")
    print(f"Colonne: {list(golden_labels_output.columns)}")

    # Salva anche statistiche dettagliate
    stats_path = output_path.replace(".csv", "_stats.txt")
    with open(stats_path, "w", encoding="utf-8") as f:
        f.write("STATISTICHE GOLDEN LABELS (TEST SET) - 7 CLASSI\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Data generazione: {pd.Timestamp.now()}\n")
        f.write(f"Numero totale di golden labels: {len(golden_labels_output)}\n\n")

        f.write("Distribuzione delle emozioni:\n")
        f.write("-" * 60 + "\n")
        for emotion, count in golden_emotion_dist.items():
            percentage = (count / len(golden_labels_output)) * 100
            f.write(f"{emotion:25s}: {count:5d} ({percentage:5.2f}%)\n")

        f.write("\nStatistiche compound scores:\n")
        f.write("-" * 60 + "\n")
        f.write(f"Media: {golden_labels_output['compound_score'].mean():.4f}\n")
        f.write(f"Mediana: {golden_labels_output['compound_score'].median():.4f}\n")
        f.write(f"Dev. Standard: {golden_labels_output['compound_score'].std():.4f}\n")
        f.write(f"Min: {golden_labels_output['compound_score'].min():.4f}\n")
        f.write(f"Max: {golden_labels_output['compound_score'].max():.4f}\n")

    print(f"\n📊 Statistiche salvate: {stats_path}")

else:
    print("\n❌ Nessun dato da salvare per le golden labels.")


TEST SET SALVATO

✅ File salvato: ../../data/processed/golden_label_sentiment_7_classes.csv
Numero di righe: 200
Colonne: ['video_name', 'caption', 'compound_score', 'emotion']

📊 Statistiche salvate: ../../data/processed/golden_label_sentiment_7_classes_stats.txt


## Fase 6: Creazione del Training/Validation Set (Senza Golden Labels)


In [6]:
# Salva il dataset escludendo le golden labels (per training/validation)
if not df_local.empty and golden_ids:
    # Filtra per escludere le golden labels
    df_without_golden = df_local[~df_local["utterance_id"].isin(golden_ids)].copy()

    # Rimuovi la colonna 'utterance_id' di supporto
    df_without_golden = df_without_golden.drop(columns=["utterance_id"])

    # Definisci il percorso di output
    output_path_without_golden = (
        "../../data/processed/asllrp_video_sentiment_data_7_classes_without_golden.csv"
    )

    # Salva il file
    df_without_golden.to_csv(output_path_without_golden, index=False)

    print(f"\n{'='*60}")
    print(f"TRAINING/VALIDATION SET SALVATO")
    print(f"{'='*60}")
    print(f"\n✅ File salvato: {output_path_without_golden}")
    print(f"Numero di righe: {len(df_without_golden)}")

    print(f"\nVerifica numerica:")
    print(f"  Righe originali: {len(df_local)}")
    print(f"  Golden Labels: {len(golden_ids)}")
    print(f"  Righe rimanenti: {len(df_without_golden)}")
    print(f"  Somma (golden + rimanenti): {len(golden_ids) + len(df_without_golden)}")

    if len(golden_ids) + len(df_without_golden) == len(df_local):
        print("  ✅ Verifica superata: Nessuna riga persa!")
    else:
        print("  ⚠️  Attenzione: Discrepanza nel numero di righe!")

    # Mostra la distribuzione delle emozioni nel training set
    print("\nDistribuzione delle emozioni nel Training/Validation Set:")
    train_emotion_dist = df_without_golden["emotion"].value_counts().sort_index()
    for emotion, count in train_emotion_dist.items():
        percentage = (count / len(df_without_golden)) * 100
        print(f"  {emotion:25s}: {count:5d} ({percentage:5.2f}%)")

    # Salva statistiche anche per il training set
    stats_path = output_path_without_golden.replace(".csv", "_stats.txt")
    with open(stats_path, "w", encoding="utf-8") as f:
        f.write("STATISTICHE TRAINING/VALIDATION SET - 7 CLASSI\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Data generazione: {pd.Timestamp.now()}\n")
        f.write(f"Numero totale di righe: {len(df_without_golden)}\n\n")

        f.write("Distribuzione delle emozioni:\n")
        f.write("-" * 60 + "\n")
        for emotion, count in train_emotion_dist.items():
            percentage = (count / len(df_without_golden)) * 100
            f.write(f"{emotion:25s}: {count:5d} ({percentage:5.2f}%)\n")

        f.write("\nStatistiche compound scores:\n")
        f.write("-" * 60 + "\n")
        f.write(f"Media: {df_without_golden['compound_score'].mean():.4f}\n")
        f.write(f"Mediana: {df_without_golden['compound_score'].median():.4f}\n")
        f.write(f"Dev. Standard: {df_without_golden['compound_score'].std():.4f}\n")
        f.write(f"Min: {df_without_golden['compound_score'].min():.4f}\n")
        f.write(f"Max: {df_without_golden['compound_score'].max():.4f}\n")

    print(f"\n📊 Statistiche salvate: {stats_path}")

else:
    print(
        "\n❌ Impossibile creare il training/validation set a causa di errori precedenti."
    )


TRAINING/VALIDATION SET SALVATO

✅ File salvato: ../../data/processed/asllrp_video_sentiment_data_7_classes_without_golden.csv
Numero di righe: 1918

Verifica numerica:
  Righe originali: 2118
  Golden Labels: 200
  Righe rimanenti: 1918
  Somma (golden + rimanenti): 2118
  ✅ Verifica superata: Nessuna riga persa!

Distribuzione delle emozioni nel Training/Validation Set:
  Extremely Negative       :     4 ( 0.21%)
  Extremely Positive       :    38 ( 1.98%)
  Negative                 :   105 ( 5.47%)
  Neutral                  :  1007 (52.50%)
  Positive                 :   364 (18.98%)
  Somewhat Negative        :   186 ( 9.70%)
  Somewhat Positive        :   214 (11.16%)

📊 Statistiche salvate: ../../data/processed/asllrp_video_sentiment_data_7_classes_without_golden_stats.txt


## Fase 7: Confronto delle Distribuzioni


In [7]:
# Confronta le distribuzioni tra test set e training set
if not golden_labels_df.empty and not df_without_golden.empty:
    import matplotlib.pyplot as plt
    import matplotlib

    matplotlib.use("Agg")

    emotions_order = [
        "Extremely Negative",
        "Negative",
        "Somewhat Negative",
        "Neutral",
        "Somewhat Positive",
        "Positive",
        "Extremely Positive",
    ]

    # Prepara i dati per il grafico
    test_counts = [golden_emotion_dist.get(emotion, 0) for emotion in emotions_order]
    train_counts = [train_emotion_dist.get(emotion, 0) for emotion in emotions_order]

    # Normalizza in percentuali
    test_percentages = [(count / len(golden_labels_df)) * 100 for count in test_counts]
    train_percentages = [
        (count / len(df_without_golden)) * 100 for count in train_counts
    ]

    # Crea il grafico
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    colors = [
        "#8B0000",
        "#DC143C",
        "#FFA07A",
        "#808080",
        "#90EE90",
        "#32CD32",
        "#006400",
    ]

    # Test Set
    ax1 = axes[0]
    bars1 = ax1.bar(range(len(emotions_order)), test_percentages, color=colors)
    ax1.set_title("Test Set (Golden Labels)", fontsize=13, fontweight="bold")
    ax1.set_xlabel("Emozione", fontsize=11)
    ax1.set_ylabel("Percentuale (%)", fontsize=11)
    ax1.set_xticks(range(len(emotions_order)))
    ax1.set_xticklabels(
        [
            "Estr.\nNeg.",
            "Neg.",
            "Po'\nNeg.",
            "Neutro",
            "Po'\nPos.",
            "Pos.",
            "Estr.\nPos.",
        ],
        fontsize=9,
    )
    ax1.grid(axis="y", alpha=0.3)

    for bar, count in zip(bars1, test_counts):
        height = bar.get_height()
        if count > 0:
            ax1.text(
                bar.get_x() + bar.get_width() / 2.0,
                height,
                f"{int(count)}\n({height:.1f}%)",
                ha="center",
                va="bottom",
                fontsize=8,
            )

    # Training/Validation Set
    ax2 = axes[1]
    bars2 = ax2.bar(range(len(emotions_order)), train_percentages, color=colors)
    ax2.set_title("Training/Validation Set", fontsize=13, fontweight="bold")
    ax2.set_xlabel("Emozione", fontsize=11)
    ax2.set_ylabel("Percentuale (%)", fontsize=11)
    ax2.set_xticks(range(len(emotions_order)))
    ax2.set_xticklabels(
        [
            "Estr.\nNeg.",
            "Neg.",
            "Po'\nNeg.",
            "Neutro",
            "Po'\nPos.",
            "Pos.",
            "Estr.\nPos.",
        ],
        fontsize=9,
    )
    ax2.grid(axis="y", alpha=0.3)

    for bar, count in zip(bars2, train_counts):
        height = bar.get_height()
        if count > 0:
            ax2.text(
                bar.get_x() + bar.get_width() / 2.0,
                height,
                f"{int(count)}\n({height:.1f}%)",
                ha="center",
                va="bottom",
                fontsize=8,
            )

    plt.tight_layout()

    # Salva il grafico
    output_folder = "../../reports/figures"
    os.makedirs(output_folder, exist_ok=True)
    comparison_path = os.path.join(
        output_folder, "test_train_distribution_comparison_7_classes.png"
    )
    plt.savefig(comparison_path, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"\n📊 Grafico di confronto salvato: {comparison_path}")


📊 Grafico di confronto salvato: ../../reports/figures/test_train_distribution_comparison_7_classes.png


## Riepilogo Finale

✅ **Processo completato con successo!**

### File generati:

1. **Test Set (Golden Labels)**:

   - `golden_label_sentiment_7_classes.csv` - Dati per la valutazione finale
   - `golden_label_sentiment_7_classes_stats.txt` - Statistiche del test set

2. **Training/Validation Set**:

   - `asllrp_video_sentiment_data_7_classes_without_golden.csv` - Dati per addestramento
   - `asllrp_video_sentiment_data_7_classes_without_golden_stats.txt` - Statistiche del training set

3. **Visualizzazioni**:
   - `test_train_distribution_comparison_7_classes.png` - Confronto distribuzioni

### Separazione dei dati:

- ✅ **Test Set**: Contiene solo le golden labels validate da EmoSign
- ✅ **Training Set**: Contiene tutti gli altri dati, senza overlap con il test set
- ✅ **Nessuna contaminazione**: Separazione netta garantita

### Prossimi passi:

1. Utilizzare il Training/Validation Set per:

   - Addestramento dei modelli
   - Cross-validation
   - Tuning degli iperparametri

2. Utilizzare il Test Set (Golden Labels) per:

   - Valutazione finale delle performance
   - Confronto con altri studi
   - Validazione robusta dei risultati

3. Considerare tecniche di data augmentation per le classi meno rappresentate

4. Applicare class weights durante il training per gestire lo sbilanciamento


## ⚠️ IMPORTANTE: Perché Solo 5 Classi su 7 nelle Golden Labels?

### 📊 Situazione Attuale:

Le golden labels contengono solo **5 delle 7 classi emotive**:

✅ **Classi Presenti:**

- Extremely Negative: 5 esempi (2.50%)
- Negative: 78 esempi (39.00%)
- Somewhat Negative: 16 esempi (8.00%)
- Positive: 71 esempi (35.50%)
- Extremely Positive: 30 esempi (15.00%)

❌ **Classi Mancanti:**

- **Neutral**: 0 esempi (0%)
- **Somewhat Positive**: 0 esempi (0%)

---

### 🤔 Spiegazione:

#### 1. **Dataset EmoSign Originale (3 Classi)**

Il dataset EmoSign da cui provengono le golden labels originariamente ha solo **3 classi** (Positive, Negative, Neutral), etichettate manualmente. Quando abbiamo applicato il nostro sistema a 7 classi basato su VADER:

- I video etichettati come "Positive" → hanno generato compound scores che li hanno classificati come **Positive** o **Extremely Positive**
- I video etichettati come "Negative" → hanno generato compound scores che li hanno classificati in varie categorie negative
- I video etichettati come "Neutral" → potrebbero non essere presenti nelle 200 corrispondenze trovate, o hanno compound scores che li hanno classificati diversamente

#### 2. **Distribuzione dei Compound Scores**

Dalle statistiche:

- **Media**: 0.1057 (appena sopra la soglia Neutral)
- **Mediana**: 0.5994 (nell'intervallo Positive)
- **Range**: da -0.9440 a 0.8910

Questo indica che:

- Il dataset è leggermente **sbilanciato verso il positivo**
- L'area "Somewhat Positive" (0.10 to 0.40) è una **zona di transizione** poco rappresentata
- L'area "Neutral" (-0.10 to 0.10) è molto **stretta** e potrebbe contenere pochi video

#### 3. **Soglie Config 2 (Concentrata)**

La configurazione scelta ha un'area Neutral più ampia (0.20) ma comunque selettiva:

```
Neutral:          -0.10 to  0.10  (area 0.20)
Somewhat Positive: 0.10 to  0.40  (area 0.30)
```

Con una media di 0.1057, molti video sono **appena sopra** la soglia Neutral, ma non tutti finiscono in "Somewhat Positive" perché molti hanno compound scores più alti.

---

### ✅ Questo è Normale?

**Sì, è completamente normale e previsto!** Ecco perché:

1. **Le golden labels derivano da un dataset con 3 classi**, non 7
2. **La classificazione a 7 classi è il nostro raffinamento**, non la ground truth originale
3. **I video validati manualmente** probabilmente avevano emozioni chiaramente positive o negative, non intermedie
4. **Le aree intermedie** (Somewhat Positive, Neutral) sono più difficili da validare manualmente

---

### 💡 Implicazioni Pratiche:

#### ✅ **Vantaggi:**

- Le golden labels sono **affidabili** per le 5 classi presenti
- Hai una **buona copertura** delle emozioni negative e positive
- Le classi estreme (Extremely Negative/Positive) sono rappresentate

#### ⚠️ **Limitazioni:**

- **Non puoi validare** le classi "Neutral" e "Somewhat Positive" con le golden labels
- **Test set parziale**: Valuta solo 5 classi su 7
- **Sbilanciamento**: Negative (78) e Positive (71) dominano

---

### 🔧 Soluzioni Possibili:

#### Opzione 1: **Accettare la Situazione** (CONSIGLIATO)

- Usare le golden labels per validare le 5 classi presenti
- Usare **validazione incrociata** sul training set per le altre 2 classi
- Riportare nei risultati che le golden labels coprono solo 5/7 classi

#### Opzione 2: **Creare Golden Labels Sintetiche**

- Validare **manualmente** alcuni video del training set nelle classi mancanti
- Aggiungere queste nuove golden labels al test set
- ⚠️ Richiede lavoro manuale significativo

#### Opzione 3: **Modificare le Soglie**

- Usare una configurazione diversa (es. Config 1 Uniforme) che potrebbe generare più classi
- ⚠️ Potrebbe ridurre la precisione semantica delle classificazioni

#### Opzione 4: **Aggregare le Classi per il Test**

- Nel test finale, aggregare:
  - Neutral + Somewhat Positive + Somewhat Negative → **"Neutral-ish"**
  - Positive + Extremely Positive → **"Positive"**
  - Negative + Extremely Negative → **"Negative"**
- Tornare a un sistema a **3 classi** per il test con golden labels
- Mantenere 7 classi per l'analisi sul training set

---

### 📝 Raccomandazione Finale:

**Accettare la situazione (Opzione 1)** è la scelta migliore perché:

1. Le golden labels rimangono affidabili e validate
2. Non introduci bias con etichette sintetiche
3. È una limitazione **comune** nei dataset di sentiment analysis
4. Puoi comunque valutare il modello su 5 classi significative
5. Usi cross-validation per le classi non coperte

Nei risultati della tesi, basta **documentare chiaramente** che:

- Le golden labels coprono 5/7 classi (200 esempi)
- Le classi Neutral e Somewhat Positive sono valutate solo sul training set
- Questo è dovuto alla natura del dataset EmoSign originale (3 classi)
